<a href="https://colab.research.google.com/github/olstjr/Allora_Chatbot_in_THU/blob/main/seoultech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playwright
!apt install tesseract-ocr
!pip install pytesseract
!playwright install
!playwright install-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 10.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,290 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124926 fil

In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import json
from bs4 import BeautifulSoup
import os
import re
import aiohttp
import cv2
import pytesseract
import numpy as np
!apt-get install -y tesseract-ocr tesseract-ocr-kor
!pip install pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  tesseract-ocr-kor
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 1,052 kB of archives.
After this operation, 1,693 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-kor all 1:4.00~git30-7274cfa-1.1 [1,052 kB]
Fetched 1,052 kB in 1s (1,290 kB/s)
Selecting previously unselected package tesseract-ocr-kor.
(Reading database ... 126609 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-kor_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-kor (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-kor (1:4.00~git30-7274cfa-1.1) ...


In [ ]:
import requests
import cv2
import numpy as np
import pytesseract
from bs4 import BeautifulSoup
import json
import time

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# ✅ 공지사항 리스트 URL (1~10 페이지 반복)
base_list_url = "https://www.seoultech.ac.kr/service/info/janghak/?do=list&searchtext=&searchtype=-1&nowpage={}"

# ✅ 공지사항 상세 URL 템플릿
detail_url_template = "https://www.seoultech.ac.kr/service/info/janghak/?do=commonview&bnum=5233&bidx={}&qidx=5233&cate=0&allboard=true&nowpage={}"

# ✅ 결과 저장 리스트
all_notices = []

# ✅ 1~10 페이지 반복
for page in range(1, 11):
    list_url = base_list_url.format(page)
    print(f"🔍 {page} 페이지에서 공지사항 목록 가져오는 중...")

    response = requests.get(list_url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")

    # 🔹 공지사항 목록에서 개별 공지 URL 찾기
    notice_links = []
    for link in soup.select("a[href*='bidx=']"):
        href = link.get("href")
        if "bidx=" in href:
            bidx = href.split("bidx=")[1].split("&")[0]
            notice_links.append(detail_url_template.format(bidx, page))

    if not notice_links:
        print(f"📌 {page} 페이지에서 공지사항을 찾을 수 없음. 다음 페이지로 이동.")
        continue

    print(f"✅ {len(notice_links)}개의 공지사항을 찾았습니다.")

    # ✅ 개별 공지사항 크롤링 및 OCR 수행
    for notice_url in notice_links:
        try:
            print(f"🔹 공지사항 크롤링 중: {notice_url}")
            notice_response = requests.get(notice_url, headers=HEADERS)
            notice_soup = BeautifulSoup(notice_response.text, "html.parser")

            # 🔹 공지 제목 추출
            title_tag = notice_soup.select_one(".wrap_view .tbl_view tr td")
            title = title_tag.text.strip() if title_tag else "제목 없음"

            # 🔹 공지 날짜 추출
            date_tag = notice_soup.select_one(".wrap_view .tbl_view tr td:nth-of-type(4)")
            date = date_tag.text.strip() if date_tag else "날짜 없음"

            # 🔹 본문 내용 추출
            content_td = notice_soup.select_one(".wrap_view .tbl_view tr td.cont")
            content = content_td.get_text("\n", strip=True) if content_td else "본문 없음"

            # 🔹 특정 이미지 찾기 (`wrap_view` 내 `tbl_view` 테이블 안에 있는 이미지)
            target_img = None
            for img in notice_soup.select(".wrap_view .tbl_view img"):
                if img.get("src", "").startswith("/storage/"):
                    target_img = img
                    break  # 첫 번째 이미지만 OCR

            # ✅ OCR 결과 저장 리스트
            image_texts = []

            # ✅ OCR 수행 (이미지가 있을 경우)
            if target_img and "src" in target_img.attrs:
                img_url = target_img["src"]

                # ✅ 절대 경로로 변환
                if img_url.startswith("/storage/"):
                    base_url = "https://www.seoultech.ac.kr"
                    img_url = base_url + img_url

                print(f"✅ OCR 대상 이미지 URL: {img_url}")

                # ✅ 이미지 다운로드 및 OCR 실행
                img_response = requests.get(img_url, headers=HEADERS)
                if img_response.status_code == 200:
                    img_array = np.asarray(bytearray(img_response.content), dtype=np.uint8)
                    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

                    # ✅ OCR 전처리 (흑백 변환 및 대비 증가)
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

                    # ✅ Tesseract OCR 실행 (한글 + 영어)
                    extracted_text = pytesseract.image_to_string(binary, lang="kor+eng").strip()

                    if extracted_text:
                        image_texts.append(extracted_text)

                else:
                    print(f"❌ 이미지 요청 실패: {img_url}, 상태 코드: {img_response.status_code}")

            # ✅ OCR 결과 본문에 추가
            if image_texts:
                content += "\n\n[이미지 OCR 추출 내용]\n" + "\n".join(image_texts)

            # ✅ 데이터 저장
            all_notices.append({
                "title": title,
                "date": date,
                "url": notice_url,
                "content": content,
                "has_images": bool(image_texts)
            })

            print(f"✅ 공지사항 크롤링 완료: {title} ({date})")

            # ✅ 요청 간격 조절 (서버 부하 방지)
            time.sleep(2)

        except Exception as e:
            print(f"⚠️ 공지사항 크롤링 실패: {notice_url} | 오류: {e}")

# ✅ JSON 파일로 저장
with open("new_seoultech_scholarship.json", "w", encoding="utf-8") as f:
    json.dump(all_notices, f, ensure_ascii=False, indent=4)

print(f"✅ JSON 저장 완료: seoultech_scholarship_notices.json")


🔍 1 페이지에서 공지사항 목록 가져오는 중...
✅ 22개의 공지사항을 찾았습니다.
🔹 공지사항 크롤링 중: https://www.seoultech.ac.kr/service/info/janghak/?do=commonview&bnum=5233&bidx=5233&qidx=5233&cate=0&allboard=true&nowpage=1
✅ 공지사항 크롤링 완료: 제목 없음 (날짜 없음)
🔹 공지사항 크롤링 중: https://www.seoultech.ac.kr/service/info/janghak/?do=commonview&bnum=5233&bidx=57138&qidx=5233&cate=0&allboard=true&nowpage=1
✅ 공지사항 크롤링 완료: 제목 없음 (날짜 없음)
🔹 공지사항 크롤링 중: https://www.seoultech.ac.kr/service/info/janghak/?do=commonview&bnum=5233&bidx=554823&qidx=5233&cate=0&allboard=true&nowpage=1
✅ 공지사항 크롤링 완료: 제목 없음 (날짜 없음)
🔹 공지사항 크롤링 중: https://www.seoultech.ac.kr/service/info/janghak/?do=commonview&bnum=5233&bidx=554686&qidx=5233&cate=0&allboard=true&nowpage=1
✅ 공지사항 크롤링 완료: 2025학년도 1학기 국가장학금 2차 신청기간 안내(어의, 봉사장학금 수혜 희망자 국가장학금 신청 필수) (날짜 없음)
🔹 공지사항 크롤링 중: https://www.seoultech.ac.kr/service/info/janghak/?do=commonview&bnum=5233&bidx=554196&qidx=5233&cate=0&allboard=true&nowpage=1
✅ 공지사항 크롤링 완료: (통정시 신청, 1.13.~1.17.) 2024-2학기 마일리지장학금 지급 신청기간 안내 (날짜 없음)
🔹 공지사항 크